In [1]:
import pandas as pd
import numpy as np
import csv
import mysql.connector 

In [2]:
data = pd.read_csv(r'data/orders.csv')

In [3]:
data.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [4]:
#Data Cleaning
#Clean Header Name
# Trim leading and trailing white space from column names
# Series.str.strip : Remove leading and trailing characters in Series/Index.
# Series.str.lstrip : Remove leading characters in Series/Index.
# Series.str.rstrip : Remove trailing characters in Series/Index.
data.columns = data.columns.str.strip()
#Remove the spacial charector from column name
data.columns = [col.lower().replace(" ", "_").replace("?", "").replace("-", "_") \
                    .replace(r"/", "_").replace("\\", "_").replace("%", "").replace("$", "") \
                    .replace(")", "").replace(r"(", "").replace(".", "") for col in data.columns]
#     print(dataframe.columns)

In [5]:
data.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   order_id          9994 non-null   int64 
 1   order_date        9994 non-null   object
 2   ship_mode         9993 non-null   object
 3   segment           9994 non-null   object
 4   country           9994 non-null   object
 5   city              9994 non-null   object
 6   state             9994 non-null   object
 7   postal_code       9994 non-null   int64 
 8   region            9994 non-null   object
 9   category          9994 non-null   object
 10  sub_category      9994 non-null   object
 11  product_id        9994 non-null   object
 12  cost_price        9994 non-null   int64 
 13  list_price        9994 non-null   int64 
 14  quantity          9994 non-null   int64 
 15  discount_percent  9994 non-null   int64 
dtypes: int64(6), object(10)
memory usage: 1.2+ MB


In [7]:
#Transform the order date into datetime format
data['order_date'] = pd.to_datetime(data['order_date'], format="%Y-%m-%d")

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          9994 non-null   int64         
 1   order_date        9994 non-null   datetime64[ns]
 2   ship_mode         9993 non-null   object        
 3   segment           9994 non-null   object        
 4   country           9994 non-null   object        
 5   city              9994 non-null   object        
 6   state             9994 non-null   object        
 7   postal_code       9994 non-null   int64         
 8   region            9994 non-null   object        
 9   category          9994 non-null   object        
 10  sub_category      9994 non-null   object        
 11  product_id        9994 non-null   object        
 12  cost_price        9994 non-null   int64         
 13  list_price        9994 non-null   int64         
 14  quantity          9994 n

In [9]:
data.to_csv('data/orders_clean.csv', index=False)

In [10]:
######Create a format query for creating table schema
replacements = {
    'object' : 'varchar(255)',
    'float64' : 'float',
    'int64'   : 'int',
    'datetime64[ns]': 'timestamp',
    'timedelta64[ns]': 'varchar(255)'
}

In [11]:
col_str = ",".join("{} {}".format(n, d) for (n, d) in zip(data.columns, data.dtypes.replace(replacements)))

### Estblashing connection with MySQL database

In [12]:
#Creating the connection with MySQL Server
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "RAJUraj@88",
    auth_plugin='mysql_native_password'  # Specify the authentication plugin
)
#Creating a cursor to execute SQL queries
mycursor = mydb.cursor()


In [13]:
#Create a new database
mycursor.execute("create database mysql_vscode")

DatabaseError: 1007 (HY000): Can't create database 'mysql_vscode'; database exists

In [14]:
#select the database
mycursor.execute("use mysql_vscode")

In [15]:
tbl_name = 'orders'
#drop table if table name is already exists
mycursor.execute("drop table if exists %s;" %(tbl_name))

#create query for create table statement
create_table_query = "create table %s (%s)" %(tbl_name, col_str)
mycursor.execute(create_table_query)

In [ ]:
Insert Data into database

In [17]:
#againg reading the clean data file
with open("data/orders_clean.csv", 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    header = next(csvreader) #skip the header and store it in a variable
    for row in csvreader:
        # print(row)
        rows = tuple(row) #Convert the list into tuple
        insert_query = "insert into %s values %s;" %(tbl_name, rows)
        mycursor.execute(insert_query)
        # break
    mydb.commit()
    # mycursor.close()
    # mydb.close()

In [18]:
mycursor.execute("show tables")

In [19]:
mycursor.fetchall()

[('orders',)]

In [20]:
mycursor.execute("select * from orders;")

In [21]:
mycursor.fetchall()

[(1,
  datetime.datetime(2023, 3, 1, 0, 0),
  'Second Class',
  'Consumer',
  'United States',
  'Henderson',
  'Kentucky',
  42420,
  'South',
  'Furniture',
  'Bookcases',
  'FUR-BO-10001798',
  240,
  260,
  2,
  2),
 (2,
  datetime.datetime(2023, 8, 15, 0, 0),
  'Second Class',
  'Consumer',
  'United States',
  'Henderson',
  'Kentucky',
  42420,
  'South',
  'Furniture',
  'Chairs',
  'FUR-CH-10000454',
  600,
  730,
  3,
  3),
 (3,
  datetime.datetime(2023, 1, 10, 0, 0),
  'Second Class',
  'Corporate',
  'United States',
  'Los Angeles',
  'California',
  90036,
  'West',
  'Office Supplies',
  'Labels',
  'OFF-LA-10000240',
  10,
  10,
  2,
  5),
 (4,
  datetime.datetime(2022, 6, 18, 0, 0),
  'Standard Class',
  'Consumer',
  'United States',
  'Fort Lauderdale',
  'Florida',
  33311,
  'South',
  'Furniture',
  'Tables',
  'FUR-TA-10000577',
  780,
  960,
  5,
  2),
 (5,
  datetime.datetime(2022, 7, 13, 0, 0),
  'Standard Class',
  'Consumer',
  'United States',
  'Fort Laude